In [1]:
import numpy as np
from transformers import AutoTokenizer
from datasets import Dataset

/opt/anaconda3/envs/brainteaser/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sentence_puzzles = np.load('data/SP-train.npy', allow_pickle=True)
word_puzzles = np.load('data/WP-train.npy', allow_pickle=True)

In [3]:
sentence_puzzles[0]

{'id': 'SP-0',
 'question': 'Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?',
 'answer': 'Each daughter shares the same brother.',
 'distractor1': 'Some daughters get married and have their own family.',
 'distractor2': 'Some brothers were not loved by family and moved away.',
 'distractor(unsure)': 'None of above.',
 'label': 1,
 'choice_list': ['Some daughters get married and have their own family.',
  'Each daughter shares the same brother.',
  'Some brothers were not loved by family and moved away.',
  'None of above.'],
 'choice_order': [1, 0, 2, 3]}

In [ ]:
choices = ['answer', 'distractor1', 'distractor2', 'distractor(unsure)']
attributes = ['question'] + choices

sentence_puzzles_dict = {key: [puzzle[key] for puzzle in sentence_puzzles] for key in attributes}
for key in sentence_puzzles_dict:
    sentence_puzzles_dict[key] = [str(value) for value in sentence_puzzles_dict[key]]

sentence_puzzles_dataset = Dataset.from_dict(sentence_puzzles_dict)

['question', 'answer', 'distractor1', 'distractor2', 'distractor(unsure)']

In [4]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')

/opt/anaconda3/envs/brainteaser/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
def preprocess_function(examples):
    questions = [[question] * 4 for question in examples['question']]
    answers = [[examples[choice][i] for choice in choices] for i in range(len(examples['answer']))]
    
    questions = sum(questions, [])
    answers = sum(answers, [])

    tokenized_examples = tokenizer(questions, answers, truncation=True)
    return {key: [value[i:i + 4] for i in range(0, len(value), 4)] for key, value in tokenized_examples.items()}

In [ ]:
examples = sentence_puzzles_dataset[:5]
features = preprocess_function(examples)

In [ ]:
idx = 0
[tokenizer.decode(features['input_ids'][idx][i]) for i in range(4)]